In [2]:
# Define some basic parameters

batch_size = 32
img_height = 150
img_width = 150

In [3]:
# Define the path to the datasets directories

train_path = '/home/sahil/Desktop/applied.ai/computer vision/folder/seg_train'
test_path = '/home/sahil/Desktop/applied.ai/computer vision/folder/seg_test'
pred_path = '/home/sahil/Desktop/applied.ai/computer vision/folder/seg_pred'

In [4]:
from keras.preprocessing import image_dataset_from_directory

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
# Define data loading function

def load_data(path, labels):
    dataset = image_dataset_from_directory(
        directory=path,
        labels=labels,
        seed=123,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )
    return dataset

In [7]:
# Load the datasets

train_ds = load_data(train_path, labels='inferred')
test_ds = load_data(test_path, labels='inferred')


Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [8]:
# Explore the image labels

class_names = train_ds.class_names
class_names

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [9]:
# Show the first nine images from the training dataset

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

In [10]:
AUTOTUNE = tf.data.AUTOTUNE

In [11]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# pred_ds = pred_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Standardize the Data
The RGB channel values are in the [0, 255] range. This is not ideal for a neural network.

The values should be standardized to be in the [0, 1] range.

In [1]:
from keras.layers import Rescaling

In [2]:
scaling = Rescaling(1. / 255)

train_ds = train_ds.map(lambda x, y: (scaling(x), y))
test_ds = test_ds.map(lambda x, y: (scaling(x), y))
# pred_ds = pred_ds.map(lambda x: scaling(x))

# Create Simple CNN Model (sCNN)
The Sequential model consists of three convolution blocks with a max pooling layer in each of them.

There's a fully-connected layer with 128 units on top of it that is activated by a ReLU activation function ('relu').

This model is a simple model and has not been tuned for high accuracy.

In [14]:
sCNN = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(6, activation='softmax')
])

# Compile the Model
The Compile method configures the model for training and validation using the optimizer, loss function, and evaluation metrics.

This workflow will use the Adam optimizer, the Sparse Categorical Crossentropy loss function, and the Accuracy evaluation metric.

In [15]:
sCNN.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [16]:
sCNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [ ]:
epochs = 1

history = sCNN.fit(train_ds, validation_data=test_ds, epochs=epochs)

 27/439 [>.............................] - ETA: 51:55 - loss: 136.2324 - accuracy: 0.1751

In [ ]:
sCNN.save("model_intel_classification")